# Tic Tac Toe

In this cookbook, I want to show how Multi-Step environments work in CAMEL. Our RL modules were built to mimic OpenAI Gym, so if you're familiar with Gym's interface, you'll feel right at home.

We will use the Tic-Tac-Toe environment as an example to show the lifecycle of an environment.

The Tic-Tac-Toe environment can be used to evaluate agents, generate synthetic data for distillation, or train an agent to play the game.

First, we need to initialize our environment and set it up. Then we can call `reset` to get our initial observation.

In [1]:
import asyncio
from camel.environments.models import Action
from camel.environments.tic_tac_toe import TicTacToeEnv, Opponent

# we can choose the playstyle of our opponent to be either 'random' or 'optimal' (computed using minimax)
opp = Opponent(play_style="random")
env = TicTacToeEnv(opponent=opp)
await env.setup()

obs = await env.reset()
print("Initial Observation:\n")
print(obs.question)

Initial Observation:

You are playing Tic Tac Toe with standard rules.
You are the player with X.
Choose a number between 1 and 9 to place an X.
This is the current state of the board:
1 | 2 | 3
---------
4 | 5 | 6
---------
7 | 8 | 9
Each number that you can see is still an empty field that you can place your 'X' in. Please end your response with <Action> [a number from 1 to 9]


We will use GPT-4o-mini, so let's enter our API key.

In [2]:
import os
from getpass import getpass

openai_api_key = getpass('Enter your API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key

Let's next define the model-backend and the agent.

You can also add a system prompt or equip your agent with tools, but for the sake of simplicity we just create a bare agent with GPT-4o-mini.

In [3]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.configs import ChatGPTConfig
from camel.agents import ChatAgent

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O_MINI,
    model_config_dict=ChatGPTConfig().as_dict(),
)

agent = ChatAgent(model=model)

Next, we will simulate one episode.

In [4]:
while not env.is_done():

    llm_response = agent.step(obs.question).msgs[0].content
    agent.reset() # clear context window

    action = Action(llm_response=llm_response)
    result = await env.step(action)

    next_obs, reward, done, info = result

    obs = next_obs

    print("\nAgent Move:", action.llm_response)
    print("Observation:")
    print(next_obs.question)
    print("Reward:", reward)
    print("Done:", done)
    print("Info:", info)


Agent Move: I'll place my X in the center position to maximize my options. 

<Action> 5
Observation:
You are playing Tic Tac Toe with standard rules.
You are the player with X.
Choose a number between 1 and 9 to place an X.
This is the current state of the board:
O | 2 | 3
---------
4 | X | 6
---------
7 | 8 | 9
Each number that you can see is still an empty field that you can place your 'X' in. Please end your response with <Action> [a number from 1 to 9]
Reward: 0.5
Done: False
Info: {'extraction_result': '5', 'step': 1, 'state': {'board': ['O', ' ', ' ', ' ', 'X', ' ', ' ', ' ', ' '], 'game_over': False, 'winner': None, 'last_move_illegal': False, 'last_move': 5, 'extraction_error': None}, 'rewards_dict': {'x_non_loss_value': 0.5}}

Agent Move: I'll place my 'X' in position 3. 

<Action> 3
Observation:
You are playing Tic Tac Toe with standard rules.
You are the player with X.
Choose a number between 1 and 9 to place an X.
This is the current state of the board:
O | 2 | X
---------

Finally, we close the environment.

In [5]:
await env.close()